In [1]:
import pandas as pd
import geopandas as gpd
import os
import gtfs_kit as gk
import osmnx as ox
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
from shapely.geometry import Polygon, LineString
import warnings
warnings.filterwarnings('ignore')
import gtfs_kit as gk
import glob
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F
from pyspark import SparkConf
import pandas as pd
import pickle
from tqdm import tqdm

In [2]:
spark = SparkSession.builder.config('spark.executor.cores', '16').config('spark.executor.memory', '80g')\
        .config("spark.sql.session.timeZone", "UTC").config('spark.driver.memory', '80g').master("local[26]")\
        .appName("wego-daily").config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC').config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
        .config("spark.sql.datetime.java8API.enabled", "true").config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config("spark.sql.autoBroadcastJoinThreshold", -1)\
        .config("spark.driver.maxResultSize", 0)\
        .config("spark.shuffle.spill", "true")\
        .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
        .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true")\
        .config("spark.ui.showConsoleProgress", "false")\
        .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

23/01/22 13:16:52 WARN Utils: Your hostname, Slade-173477 resolves to a loopback address: 127.0.1.1; using 10.2.219.10 instead (on interface eno1)
23/01/22 13:16:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/22 13:16:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/22 13:16:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
apc_data_path = 'data/apc_weather_gtfs_20221216.parquet'
apc_data = spark.read.load(apc_data_path)
apc_data.createOrReplaceTempView("apc")
apc_data.columns

['gtfs_date',
 'dayofweek',
 'hour',
 'gtfs_route_id',
 'gtfs_direction_id',
 'stop_id',
 'transit_date',
 'trip_id',
 'day',
 'overload_id',
 'vehicle_id',
 'block_abbr',
 'activation_date',
 'activation_date_str',
 'actual_hdwy',
 'arrival_time',
 'arrival_time_str',
 'block_stop_order',
 'deactivation_date',
 'deactivation_date_str',
 'delay_time',
 'departure_time',
 'departure_time_str',
 'dwell_time',
 'is_bunched',
 'is_gapped',
 'is_target',
 'load',
 'load_factor',
 'map_latitude',
 'map_longitude',
 'offs',
 'ons',
 'pattern_num',
 'prev_depart',
 'prev_sched',
 'route_direction_name',
 'route_id',
 'sched_hdwy',
 'scheduled_time',
 'scheduled_time_str',
 'source_pattern_id',
 'stop_id_list',
 'stop_id_original',
 'stop_name',
 'stop_sequence',
 'stop_sequence_list',
 'transit_date_str',
 'update_date',
 'vehicle_capacity',
 'zero_load_at_trip_end',
 'count',
 'darksky_temperature',
 'darksky_humidity',
 'darksky_nearest_storm_distance',
 'darksky_precipitation_intensity',
 '

In [4]:
col_earlist_date = F.min('transit_date').alias('earliest')
col_latest_date = F.max('transit_date').alias('latest')
df_result = apc_data.select(col_earlist_date, col_latest_date)
df_result.show()

+-------------------+-------------------+
|           earliest|             latest|
+-------------------+-------------------+
|2020-01-01 00:00:00|2022-11-11 00:00:00|
+-------------------+-------------------+



In [ ]:
# Generate travel time to next stop data
apc_data_path = 'data/apc_weather_gtfs_20221216.parquet'
apc_data = spark.read.load(apc_data_path)
apc_data.createOrReplaceTempView("apc")

get_columns = ['transit_date', 'route_id', 'trip_id', 'stop_id_original', 'stop_sequence', 'load', 'arrival_time', 'departure_time', 'scheduled_time']
get_str = ",".join([c for c in get_columns])

query = f"""
        SELECT {get_str}
        FROM apc
        WHERE transit_date < '2022-05-01'
        """
        
apc_data = spark.sql(query)
apc_data = apc_data.filter(F.col("load") >= 0)
apc_data = apc_data.orderBy(F.col("transit_date"))
df = apc_data.toPandas()
df.groupby(['transit_date', 'trip_id']).ngroups
df_arr = []
i = 0
curr_month = set()
for (transit_date, trip_id), trip_df in df.groupby(['transit_date', 'trip_id']):
    tdf = trip_df.sort_values(['stop_sequence'])
    tdf['next_stop_id'] = tdf['stop_id_original'].shift(-1)
    tdf['next_arrival'] = tdf['arrival_time'].shift(-1)
    tdf['tt_to_next_stop'] = tdf['next_arrival'] - tdf['departure_time']
    tdf['tt_to_next_stop'] = tdf['tt_to_next_stop'].dt.total_seconds()
    df_arr.append(tdf)
    x = str(transit_date.year) + '_' + str(transit_date.month)
    if x not in curr_month:
        print(x)
        curr_month.add(x)
    i+=1
    if i == 3:
        break
_df = pd.concat(df_arr)
_df = _df.drop('arrival_time', axis=1)
_df = _df.reset_index(drop=True)
# _df.to_parquet('data/tt_to_next_stop_2022_04_31.parquet')

# Generate the travel time factors using the `merging.ipynb`